In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("TeddyNote-Parser")

LangSmith 추적을 시작합니다.
[프로젝트명]
TeddyNote-Parser


In [ ]:
!pip install -qU langchain-teddynote

## 저장할 State 정의

In [2]:
from typing import TypedDict


# GraphState 상태를 저장하는 용도로 사용합니다.
class GraphState(TypedDict):
    filepath: str  # path
    filetype: str  # pdf
    page_numbers: list[int]  # page numbers
    batch_size: int  # batch size
    split_filepaths: list[str]  # split files
    analyzed_files: list[str]  # analyzed files
    page_elements: dict[int, dict[str, list[dict]]]  # page elements
    page_metadata: dict[int, dict]  # page metadata
    page_summary: dict[int, str]  # page summary
    images: list[str]  # image paths
    images_summary: list[str]  # image summary
    tables: list[str]  # table
    tables_summary: dict[int, str]  # table summary
    texts: list[str]  # text
    texts_summary: list[str]  # text summary

## 문서를 배치 단위로 분할 (10 page)

In [3]:
import os
import pymupdf
import json
import requests
from PIL import Image

In [4]:
def split_pdf(state: GraphState):
    """
    입력 PDF를 여러 개의 작은 PDF 파일로 분할합니다.

    :param state: GraphState 객체, PDF 파일 경로와 배치 크기 정보를 포함
    :return: 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체
    """
    # PDF 파일 경로와 배치 크기 추출
    filepath = state["filepath"]
    batch_size = state["batch_size"]

    # PDF 파일 열기
    input_pdf = pymupdf.open(filepath)
    num_pages = len(input_pdf)
    print(f"총 페이지 수: {num_pages}")

    ret = []
    # PDF 분할 작업 시작
    for start_page in range(0, num_pages, batch_size):
        # 배치의 마지막 페이지 계산 (전체 페이지 수를 초과하지 않도록)
        end_page = min(start_page + batch_size, num_pages) - 1

        # 분할된 PDF 파일명 생성
        input_file_basename = os.path.splitext(filepath)[0]
        output_file = f"{input_file_basename}_{start_page:04d}_{end_page:04d}.pdf"
        print(f"분할 PDF 생성: {output_file}")

        # 새로운 PDF 파일 생성 및 페이지 삽입
        with pymupdf.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
            ret.append(output_file)

    # 원본 PDF 파일 닫기
    input_pdf.close()

    # 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체 반환
    return GraphState(split_filepaths=ret)

In [5]:
state = GraphState(filepath="data/sample-report.pdf", batch_size=10)
state_out = split_pdf(state)
state.update(state_out)
state

총 페이지 수: 22
분할 PDF 생성: data/sample-report_0000_0009.pdf
분할 PDF 생성: data/sample-report_0010_0019.pdf
분할 PDF 생성: data/sample-report_0020_0021.pdf


{'filepath': 'data/sample-report.pdf',
 'batch_size': 10,
 'split_filepaths': ['data/sample-report_0000_0009.pdf',
  'data/sample-report_0010_0019.pdf',
  'data/sample-report_0020_0021.pdf']}

## Layout Analyzer 를 사용하여 문서를 element 단위로 분할

In [6]:
class LayoutAnalyzer:
    def __init__(self, api_key):
        """
        LayoutAnalyzer 클래스의 생성자

        :param api_key: Upstage API 인증을 위한 API 키
        """
        self.api_key = api_key

    def _upstage_layout_analysis(self, input_file):
        """
        Upstage의 레이아웃 분석 API를 호출하여 문서 분석을 수행합니다.

        :param input_file: 분석할 PDF 파일의 경로
        :return: 분석 결과가 저장된 JSON 파일의 경로
        """
        # API 요청 헤더 설정
        headers = {"Authorization": f"Bearer {self.api_key}"}

        # API 요청 데이터 설정 (OCR 비활성화)
        data = {"ocr": False}

        # 분석할 PDF 파일 열기
        files = {"document": open(input_file, "rb")}

        # API 요청 보내기
        response = requests.post(
            "https://api.upstage.ai/v1/document-ai/layout-analysis",
            headers=headers,
            data=data,
            files=files,
        )

        # API 응답 처리 및 결과 저장
        if response.status_code == 200:
            # 분석 결과를 저장할 JSON 파일 경로 생성
            output_file = os.path.splitext(input_file)[0] + ".json"

            # 분석 결과를 JSON 파일로 저장
            with open(output_file, "w") as f:
                json.dump(response.json(), f, ensure_ascii=False)

            return output_file
        else:
            # API 요청이 실패한 경우 예외 발생
            raise ValueError(f"API 요청 실패. 상태 코드: {response.status_code}")

    def execute(self, input_file):
        """
        주어진 입력 파일에 대해 레이아웃 분석을 실행합니다.

        :param input_file: 분석할 PDF 파일의 경로
        :return: 분석 결과가 저장된 JSON 파일의 경로
        """
        return self._upstage_layout_analysis(input_file)

In [7]:
def analyze_layout(state: GraphState):
    # 분할된 PDF 파일 목록을 가져옵니다.
    split_files = state["split_filepaths"]

    # LayoutAnalyzer 객체를 생성합니다. API 키는 환경 변수에서 가져옵니다.
    analyzer = LayoutAnalyzer(os.environ.get("UPSTAGE_API_KEY"))

    # 분석된 파일들의 경로를 저장할 리스트를 초기화합니다.
    analyzed_files = []

    # 각 분할된 PDF 파일에 대해 레이아웃 분석을 수행합니다.
    for file in split_files:
        # 레이아웃 분석을 실행하고 결과 파일 경로를 리스트에 추가합니다.
        analyzed_files.append(analyzer.execute(file))

    # 분석된 파일 경로들을 정렬하여 새로운 GraphState 객체를 생성하고 반환합니다.
    # 정렬은 파일들의 순서를 유지하기 위해 수행됩니다.
    return GraphState(analyzed_files=sorted(analyzed_files))

In [8]:
state_out = analyze_layout(state)
state.update(state_out)
state

{'filepath': 'data/sample-report.pdf',
 'batch_size': 10,
 'split_filepaths': ['data/sample-report_0000_0009.pdf',
  'data/sample-report_0010_0019.pdf',
  'data/sample-report_0020_0021.pdf'],
 'analyzed_files': ['data/sample-report_0000_0009.json',
  'data/sample-report_0010_0019.json',
  'data/sample-report_0020_0021.json']}

## 페이지 메타데이터 추출

In [9]:
import re


def extract_start_end_page(filename):
    """
    파일 이름에서 시작 페이지와 끝 페이지 번호를 추출하는 함수입니다.

    :param filename: 분석할 파일의 이름
    :return: 시작 페이지 번호와 끝 페이지 번호를 튜플로 반환
    """
    # 파일 경로에서 파일 이름만 추출
    file_name = os.path.basename(filename)
    # 파일 이름을 '_' 기준으로 분리
    file_name_parts = file_name.split("_")

    if len(file_name_parts) >= 3:
        # 파일 이름의 뒤에서 두 번째 부분에서 숫자를 추출하여 시작 페이지로 설정
        start_page = int(re.findall(r"(\d+)", file_name_parts[-2])[0])
        # 파일 이름의 마지막 부분에서 숫자를 추출하여 끝 페이지로 설정
        end_page = int(re.findall(r"(\d+)", file_name_parts[-1])[0])
    else:
        # 파일 이름 형식이 예상과 다를 경우 기본값 설정
        start_page, end_page = 0, 0

    return start_page, end_page


def extract_page_metadata(state: GraphState):
    """
    분석된 JSON 파일들에서 페이지 메타데이터를 추출하는 함수입니다.

    :param state: 현재의 GraphState 객체
    :return: 페이지 메타데이터가 추가된 새로운 GraphState 객체
    """
    # 분석된 JSON 파일 목록 가져오기
    json_files = state["analyzed_files"]

    # 페이지 메타데이터를 저장할 딕셔너리 초기화
    page_metadata = dict()

    for json_file in json_files:
        # JSON 파일 열기 및 데이터 로드
        with open(json_file, "r") as f:
            data = json.load(f)

        # 파일명에서 시작 페이지 번호 추출
        start_page, _ = extract_start_end_page(json_file)

        # JSON 데이터에서 각 페이지의 메타데이터 추출
        for element in data["metadata"]["pages"]:
            # 원본 페이지 번호
            original_page = int(element["page"])
            # 상대적 페이지 번호 계산 (전체 문서 기준)
            relative_page = start_page + original_page - 1

            # 페이지 크기 정보 추출
            metadata = {
                "size": [
                    int(element["width"]),
                    int(element["height"]),
                ],
            }
            # 상대적 페이지 번호를 키로 하여 메타데이터 저장
            page_metadata[relative_page] = metadata

    # 추출된 페이지 메타데이터로 새로운 GraphState 객체 생성 및 반환
    return GraphState(page_metadata=page_metadata)

페이지별 사이즈를 추출합니다.

In [10]:
state_out = extract_page_metadata(state)
state.update(state_out)
state["page_metadata"]

{0: {'size': [1241, 1754]},
 1: {'size': [1241, 1754]},
 2: {'size': [1241, 1754]},
 3: {'size': [1241, 1754]},
 4: {'size': [1241, 1754]},
 5: {'size': [1241, 1754]},
 6: {'size': [1241, 1754]},
 7: {'size': [1241, 1754]},
 8: {'size': [1241, 1754]},
 9: {'size': [1241, 1754]},
 10: {'size': [1241, 1754]},
 11: {'size': [1241, 1754]},
 12: {'size': [1241, 1754]},
 13: {'size': [1241, 1754]},
 14: {'size': [1241, 1754]},
 15: {'size': [1241, 1754]},
 16: {'size': [1241, 1754]},
 17: {'size': [1241, 1754]},
 18: {'size': [1241, 1754]},
 19: {'size': [1241, 1754]},
 20: {'size': [1241, 1754]},
 21: {'size': [1241, 1754]}}

## 페이지별 HTML Element 추출

In [11]:
def extract_page_elements(state: GraphState):
    # 분석된 JSON 파일 목록을 가져옵니다.
    json_files = state["analyzed_files"]

    # 페이지별 요소를 저장할 딕셔너리를 초기화합니다.
    page_elements = dict()

    # 전체 문서에서 고유한 요소 ID를 부여하기 위한 카운터입니다.
    element_id = 0

    # 각 JSON 파일을 순회하며 처리합니다.
    for json_file in json_files:
        # 파일명에서 시작 페이지 번호를 추출합니다.
        start_page, _ = extract_start_end_page(json_file)

        # JSON 파일을 열어 데이터를 로드합니다.
        with open(json_file, "r") as f:
            data = json.load(f)

        # JSON 데이터의 각 요소를 처리합니다.
        for element in data["elements"]:
            # 원본 페이지 번호를 정수로 변환합니다.
            original_page = int(element["page"])
            # 전체 문서 기준의 상대적 페이지 번호를 계산합니다.
            relative_page = start_page + original_page - 1

            # 해당 페이지의 요소 리스트가 없으면 새로 생성합니다.
            if relative_page not in page_elements:
                page_elements[relative_page] = []

            # 요소에 고유 ID를 부여합니다.
            element["id"] = element_id
            element_id += 1

            # 요소의 페이지 번호를 상대적 페이지 번호로 업데이트합니다.
            element["page"] = relative_page
            # 요소를 해당 페이지의 리스트에 추가합니다.
            page_elements[relative_page].append(element)

    # 추출된 페이지별 요소 정보로 새로운 GraphState 객체를 생성하여 반환합니다.
    return GraphState(page_elements=page_elements)

In [12]:
state_out = extract_page_elements(state)
state.update(state_out)
state["page_elements"][0]

[{'bounding_box': [{'x': 815, 'y': 154},
   {'x': 1148, 'y': 154},
   {'x': 1148, 'y': 209},
   {'x': 815, 'y': 209}],
  'category': 'heading1',
  'html': "<h1 id='0' style='font-size:22px'>Argus Bitumen</h1>",
  'id': 0,
  'page': 0,
  'text': 'Argus Bitumen'},
 {'bounding_box': [{'x': 398, 'y': 218},
   {'x': 1151, 'y': 218},
   {'x': 1151, 'y': 273},
   {'x': 398, 'y': 273}],
  'category': 'paragraph',
  'html': "<br><p id='1' data-category='paragraph' style='font-size:20px'>Europe, Africa, Middle East and Asia-Pacific prices and commentary<br>Incorporating Argus Asphalt Report</p>",
  'id': 1,
  'page': 0,
  'text': 'Europe, Africa, Middle East and Asia-Pacific prices and commentary\nIncorporating Argus Asphalt Report'},
 {'bounding_box': [{'x': 779, 'y': 283},
   {'x': 1150, 'y': 283},
   {'x': 1150, 'y': 311},
   {'x': 779, 'y': 311}],
  'category': 'paragraph',
  'html': "<br><p id='2' data-category='paragraph' style='font-size:18px'>Issue 21-39 | Friday 1 October 2021</p>",
  '

추출된 페이지를 확인

In [ ]:
state["page_elements"].keys()

## 각 페이지의 태그를 분해합니다.

- images, tables, text 를 분해합니다.

In [17]:
def extract_tag_elements_per_page(state: GraphState):
    # GraphState 객체에서 페이지 요소들을 가져옵니다.
    page_elements = state["page_elements"]

    # 파싱된 페이지 요소들을 저장할 새로운 딕셔너리를 생성합니다.
    parsed_page_elements = dict()

    # 각 페이지와 해당 페이지의 요소들을 순회합니다.
    for key, page_element in page_elements.items():
        # 이미지, 테이블, 텍스트 요소들을 저장할 리스트를 초기화합니다.
        image_elements = []
        table_elements = []
        text_elements = []

        # 페이지의 각 요소를 순회하며 카테고리별로 분류합니다.
        for element in page_element:
            if element["category"] == "figure":
                # 이미지 요소인 경우 image_elements 리스트에 추가합니다.
                image_elements.append(element)
            elif element["category"] == "table":
                # 테이블 요소인 경우 table_elements 리스트에 추가합니다.
                table_elements.append(element)
            else:
                # 그 외의 요소는 모두 텍스트 요소로 간주하여 text_elements 리스트에 추가합니다.
                text_elements.append(element)

        # 분류된 요소들을 페이지 키와 함께 새로운 딕셔너리에 저장합니다.
        parsed_page_elements[key] = {
            "image_elements": image_elements,
            "table_elements": table_elements,
            "text_elements": text_elements,
            "elements": page_element,  # 원본 페이지 요소도 함께 저장합니다.
        }

    # 파싱된 페이지 요소들을 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(page_elements=parsed_page_elements)

In [18]:
state_out = extract_tag_elements_per_page(state)
state.update(state_out)

In [19]:
state["page_elements"].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

In [20]:
state["page_elements"][2].keys()

dict_keys(['image_elements', 'table_elements', 'text_elements', 'elements'])

In [21]:
state["page_elements"][2]["image_elements"]

[{'bounding_box': [{'x': 90, 'y': 1229},
   {'x': 598, 'y': 1229},
   {'x': 598, 'y': 1557},
   {'x': 90, 'y': 1557}],
  'category': 'figure',
  'html': '<figure><img id=\'52\' style=\'font-size:14px\' alt="Southern UK domestic Rotterdam domestic\n550\n500\n450 hhhhhhhhhhh\n400\n350\n22 Jan 16 Apr 9 Jul 1 Oct" data-coord="top-left:(89,1228); bottom-right:(597,1557)" /></figure>',
  'id': 52,
  'page': 2,
  'text': 'Southern UK domestic Rotterdam domestic\n550\n500\n450 hhhhhhhhhhh\n400\n350\n22 Jan 16 Apr 9 Jul 1 Oct'}]

In [22]:
state_out["page_elements"][2]["table_elements"]

[{'bounding_box': [{'x': 632, 'y': 261},
   {'x': 1147, 'y': 261},
   {'x': 1147, 'y': 1022},
   {'x': 632, 'y': 1022}],
  'category': 'table',
  'html': '<br><table id=\'50\' style=\'font-size:14px\'><tr><td></td><td colspan="3">€/t</td><td colspan="3">$/t</td></tr><tr><td></td><td>Low</td><td>high</td><td>±</td><td>Low</td><td>high</td><td>±</td></tr><tr><td>Domestic prices, ex-works</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Southern UK £/t</td><td>380</td><td>390</td><td>+20.00</td><td>516</td><td>529</td><td>+22.50</td></tr><tr><td>Rotterdam, Netherlands</td><td>420</td><td>435</td><td>+20.00</td><td>490</td><td>507</td><td>+20.50</td></tr><tr><td>Antwerp, Belgium</td><td>420</td><td>435</td><td>+20.00</td><td>490</td><td>507</td><td>+20.50</td></tr><tr><td>Northern Germany</td><td>385</td><td>395</td><td>+10.00</td><td>449</td><td>460</td><td>+8.50</td></tr><tr><td>Northeast Germany</td><td>350</td><td>360</td><td>+10.00</td><td>408</td><td>420</td><td

In [23]:
state["page_elements"][2]["text_elements"]

[{'bounding_box': [{'x': 91, 'y': 83},
   {'x': 222, 'y': 83},
   {'x': 222, 'y': 106},
   {'x': 91, 'y': 106}],
  'category': 'header',
  'html': "<header id='38' style='font-size:18px'>Argus Bitumen</header>",
  'id': 38,
  'page': 2,
  'text': 'Argus Bitumen'},
 {'bounding_box': [{'x': 786, 'y': 79},
   {'x': 1150, 'y': 79},
   {'x': 1150, 'y': 108},
   {'x': 786, 'y': 108}],
  'category': 'header',
  'html': "<br><header id='39' style='font-size:20px'>Issue 21-39 | Friday 1 October 2021</header>",
  'id': 39,
  'page': 2,
  'text': 'Issue 21-39 | Friday 1 October 2021'},
 {'bounding_box': [{'x': 91, 'y': 182},
   {'x': 671, 'y': 182},
   {'x': 671, 'y': 210},
   {'x': 91, 'y': 210}],
  'category': 'paragraph',
  'html': "<p id='40' data-category='paragraph' style='font-size:22px'>North aNd CeNtral europe market CommeNtary</p>",
  'id': 40,
  'page': 2,
  'text': 'North aNd CeNtral europe market CommeNtary'},
 {'bounding_box': [{'x': 91, 'y': 239},
   {'x': 196, 'y': 239},
   {'x': 

## 페이지 번호를 추출합니다.

In [24]:
def page_numbers(state: GraphState):
    return GraphState(page_numbers=list(state["page_elements"].keys()))


state_out = page_numbers(state)
state.update(state_out)
state["page_numbers"]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

## 이미지를 추출합니다.

In [25]:
class ImageCropper:
    @staticmethod
    def pdf_to_image(pdf_file, page_num, dpi=300):
        """
        PDF 파일의 특정 페이지를 이미지로 변환하는 메서드

        :param page_num: 변환할 페이지 번호 (1부터 시작)
        :param dpi: 이미지 해상도 (기본값: 300)
        :return: 변환된 이미지 객체
        """
        with pymupdf.open(pdf_file) as doc:
            page = doc[page_num].get_pixmap(dpi=dpi)
            target_page_size = [page.width, page.height]
            page_img = Image.frombytes("RGB", target_page_size, page.samples)
        return page_img

    @staticmethod
    def normalize_coordinates(coordinates, output_page_size):
        """
        좌표를 정규화하는 정적 메서드

        :param coordinates: 원본 좌표 리스트
        :param output_page_size: 출력 페이지 크기 [너비, 높이]
        :return: 정규화된 좌표 (x1, y1, x2, y2)
        """
        x_values = [coord["x"] for coord in coordinates]
        y_values = [coord["y"] for coord in coordinates]
        x1, y1, x2, y2 = min(x_values), min(y_values), max(x_values), max(y_values)

        return (
            x1 / output_page_size[0],
            y1 / output_page_size[1],
            x2 / output_page_size[0],
            y2 / output_page_size[1],
        )

    @staticmethod
    def crop_image(img, coordinates, output_file):
        """
        이미지를 주어진 좌표에 따라 자르고 저장하는 정적 메서드

        :param img: 원본 이미지 객체
        :param coordinates: 정규화된 좌표 (x1, y1, x2, y2)
        :param output_file: 저장할 파일 경로
        """
        img_width, img_height = img.size
        x1, y1, x2, y2 = [
            int(coord * dim)
            for coord, dim in zip(coordinates, [img_width, img_height] * 2)
        ]
        cropped_img = img.crop((x1, y1, x2, y2))
        cropped_img.save(output_file)

In [27]:
sample_table = state["page_elements"][2]["table_elements"][0]
sample_table

{'bounding_box': [{'x': 632, 'y': 261},
  {'x': 1147, 'y': 261},
  {'x': 1147, 'y': 1022},
  {'x': 632, 'y': 1022}],
 'category': 'table',
 'html': '<br><table id=\'50\' style=\'font-size:14px\'><tr><td></td><td colspan="3">€/t</td><td colspan="3">$/t</td></tr><tr><td></td><td>Low</td><td>high</td><td>±</td><td>Low</td><td>high</td><td>±</td></tr><tr><td>Domestic prices, ex-works</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Southern UK £/t</td><td>380</td><td>390</td><td>+20.00</td><td>516</td><td>529</td><td>+22.50</td></tr><tr><td>Rotterdam, Netherlands</td><td>420</td><td>435</td><td>+20.00</td><td>490</td><td>507</td><td>+20.50</td></tr><tr><td>Antwerp, Belgium</td><td>420</td><td>435</td><td>+20.00</td><td>490</td><td>507</td><td>+20.50</td></tr><tr><td>Northern Germany</td><td>385</td><td>395</td><td>+10.00</td><td>449</td><td>460</td><td>+8.50</td></tr><tr><td>Northeast Germany</td><td>350</td><td>360</td><td>+10.00</td><td>408</td><td>420</td><td>+9.00

In [28]:
sample_image = state["page_elements"][2]["image_elements"][0]
sample_image

{'bounding_box': [{'x': 90, 'y': 1229},
  {'x': 598, 'y': 1229},
  {'x': 598, 'y': 1557},
  {'x': 90, 'y': 1557}],
 'category': 'figure',
 'html': '<figure><img id=\'52\' style=\'font-size:14px\' alt="Southern UK domestic Rotterdam domestic\n550\n500\n450 hhhhhhhhhhh\n400\n350\n22 Jan 16 Apr 9 Jul 1 Oct" data-coord="top-left:(89,1228); bottom-right:(597,1557)" /></figure>',
 'id': 52,
 'page': 2,
 'text': 'Southern UK domestic Rotterdam domestic\n550\n500\n450 hhhhhhhhhhh\n400\n350\n22 Jan 16 Apr 9 Jul 1 Oct'}

In [29]:
def crop_image(state: GraphState):
    """
    PDF 파일에서 이미지를 추출하고 크롭하는 함수

    :param state: GraphState 객체
    :return: 크롭된 이미지 정보가 포함된 GraphState 객체
    """
    pdf_file = state["filepath"]  # PDF 파일 경로
    page_numbers = state["page_numbers"]  # 처리할 페이지 번호 목록
    output_folder = os.path.splitext(pdf_file)[0]  # 출력 폴더 경로 설정
    os.makedirs(output_folder, exist_ok=True)  # 출력 폴더 생성

    cropped_images = dict()  # 크롭된 이미지 정보를 저장할 딕셔너리
    for page_num in page_numbers:
        pdf_image = ImageCropper.pdf_to_image(
            pdf_file, page_num
        )  # PDF 페이지를 이미지로 변환
        for element in state["page_elements"][page_num]["image_elements"]:
            if element["category"] == "figure":
                # 이미지 요소의 좌표를 정규화
                normalized_coordinates = ImageCropper.normalize_coordinates(
                    element["bounding_box"], state["page_metadata"][page_num]["size"]
                )

                # 크롭된 이미지 저장 경로 설정
                output_file = os.path.join(output_folder, f"{element['id']}.png")
                # 이미지 크롭 및 저장
                ImageCropper.crop_image(pdf_image, normalized_coordinates, output_file)
                cropped_images[element["id"]] = output_file
                print(f"page:{page_num}, id:{element['id']}, path: {output_file}")
    return GraphState(
        images=cropped_images
    )  # 크롭된 이미지 정보를 포함한 GraphState 반환


def crop_table(state: GraphState):
    """
    PDF 파일에서 표를 추출하고 크롭하는 함수

    :param state: GraphState 객체
    :return: 크롭된 표 이미지 정보가 포함된 GraphState 객체
    """
    pdf_file = state["filepath"]  # PDF 파일 경로
    page_numbers = state["page_numbers"]  # 처리할 페이지 번호 목록
    output_folder = os.path.splitext(pdf_file)[0]  # 출력 폴더 경로 설정
    os.makedirs(output_folder, exist_ok=True)  # 출력 폴더 생성

    cropped_images = dict()  # 크롭된 표 이미지 정보를 저장할 딕셔너리
    for page_num in page_numbers:
        pdf_image = ImageCropper.pdf_to_image(
            pdf_file, page_num
        )  # PDF 페이지를 이미지로 변환
        for element in state["page_elements"][page_num]["table_elements"]:
            if element["category"] == "table":
                # 표 요소의 좌표를 정규화
                normalized_coordinates = ImageCropper.normalize_coordinates(
                    element["bounding_box"], state["page_metadata"][page_num]["size"]
                )

                # 크롭된 표 이미지 저장 경로 설정
                output_file = os.path.join(output_folder, f"{element['id']}.png")
                # 표 이미지 크롭 및 저장
                ImageCropper.crop_image(pdf_image, normalized_coordinates, output_file)
                cropped_images[element["id"]] = output_file
                print(f"page:{page_num}, id:{element['id']}, path: {output_file}")
    return GraphState(
        tables=cropped_images
    )  # 크롭된 표 이미지 정보를 포함한 GraphState 반환

In [30]:
state["page_elements"].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

In [31]:
state_out = crop_image(state)
state.update(state_out)
state["images"]
# state.update(state6)

page:0, id:15, path: data/sample-report/15.png
page:1, id:23, path: data/sample-report/23.png
page:1, id:29, path: data/sample-report/29.png
page:2, id:52, path: data/sample-report/52.png
page:3, id:68, path: data/sample-report/68.png
page:4, id:105, path: data/sample-report/105.png
page:5, id:126, path: data/sample-report/126.png
page:6, id:138, path: data/sample-report/138.png
page:8, id:193, path: data/sample-report/193.png
page:9, id:218, path: data/sample-report/218.png
page:10, id:251, path: data/sample-report/251.png
page:11, id:268, path: data/sample-report/268.png
page:13, id:300, path: data/sample-report/300.png
page:18, id:404, path: data/sample-report/404.png
page:19, id:424, path: data/sample-report/424.png
page:20, id:458, path: data/sample-report/458.png
page:20, id:470, path: data/sample-report/470.png
page:21, id:477, path: data/sample-report/477.png
page:21, id:501, path: data/sample-report/501.png


{15: 'data/sample-report/15.png',
 23: 'data/sample-report/23.png',
 29: 'data/sample-report/29.png',
 52: 'data/sample-report/52.png',
 68: 'data/sample-report/68.png',
 105: 'data/sample-report/105.png',
 126: 'data/sample-report/126.png',
 138: 'data/sample-report/138.png',
 193: 'data/sample-report/193.png',
 218: 'data/sample-report/218.png',
 251: 'data/sample-report/251.png',
 268: 'data/sample-report/268.png',
 300: 'data/sample-report/300.png',
 404: 'data/sample-report/404.png',
 424: 'data/sample-report/424.png',
 458: 'data/sample-report/458.png',
 470: 'data/sample-report/470.png',
 477: 'data/sample-report/477.png',
 501: 'data/sample-report/501.png'}

In [32]:
state_out = crop_table(state)
state.update(state_out)
state["tables"]

page:0, id:12, path: data/sample-report/12.png
page:0, id:18, path: data/sample-report/18.png
page:1, id:32, path: data/sample-report/32.png
page:1, id:33, path: data/sample-report/33.png
page:2, id:50, path: data/sample-report/50.png
page:2, id:54, path: data/sample-report/54.png
page:2, id:55, path: data/sample-report/55.png
page:5, id:121, path: data/sample-report/121.png
page:5, id:122, path: data/sample-report/122.png
page:5, id:125, path: data/sample-report/125.png
page:8, id:196, path: data/sample-report/196.png
page:8, id:197, path: data/sample-report/197.png
page:11, id:270, path: data/sample-report/270.png
page:12, id:276, path: data/sample-report/276.png
page:12, id:278, path: data/sample-report/278.png
page:13, id:302, path: data/sample-report/302.png
page:13, id:303, path: data/sample-report/303.png
page:14, id:317, path: data/sample-report/317.png
page:15, id:348, path: data/sample-report/348.png
page:15, id:351, path: data/sample-report/351.png
page:17, id:386, path: dat

{12: 'data/sample-report/12.png',
 18: 'data/sample-report/18.png',
 32: 'data/sample-report/32.png',
 33: 'data/sample-report/33.png',
 50: 'data/sample-report/50.png',
 54: 'data/sample-report/54.png',
 55: 'data/sample-report/55.png',
 121: 'data/sample-report/121.png',
 122: 'data/sample-report/122.png',
 125: 'data/sample-report/125.png',
 196: 'data/sample-report/196.png',
 197: 'data/sample-report/197.png',
 270: 'data/sample-report/270.png',
 276: 'data/sample-report/276.png',
 278: 'data/sample-report/278.png',
 302: 'data/sample-report/302.png',
 303: 'data/sample-report/303.png',
 317: 'data/sample-report/317.png',
 348: 'data/sample-report/348.png',
 351: 'data/sample-report/351.png',
 386: 'data/sample-report/386.png'}

## 페이지 텍스트 추출

In [33]:
def extract_page_text(state: GraphState):
    # 상태 객체에서 페이지 번호 목록을 가져옵니다.
    page_numbers = state["page_numbers"]

    # 추출된 텍스트를 저장할 딕셔너리를 초기화합니다.
    extracted_texts = dict()

    # 각 페이지 번호에 대해 반복합니다.
    for page_num in page_numbers:
        # 현재 페이지의 텍스트를 저장할 빈 문자열을 초기화합니다.
        extracted_texts[page_num] = ""

        # 현재 페이지의 모든 텍스트 요소에 대해 반복합니다.
        for element in state["page_elements"][page_num]["text_elements"]:
            # 각 텍스트 요소의 내용을 현재 페이지의 텍스트에 추가합니다.
            extracted_texts[page_num] += element["text"]

    # 추출된 텍스트를 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(texts=extracted_texts)

In [34]:
state_out = extract_page_text(state)
state.update(state_out)
state["texts"]

{0: 'Argus BitumenEurope, Africa, Middle East and Asia-Pacific prices and commentary\nIncorporating Argus Asphalt ReportIssue 21-39 | Friday 1 October 2021SummaryBitumen cargo values gained across Europe and Asia-Pacific,\nboosted by a jump in crude and fuel oil values.Domestic truck prices across Europe also mostly\nstrengthened as October started.Cargo premiums to high-sulphur fuel oil (HSFO) in Rot-\nterdam and the Baltics fell by $2-3/t, but outright values\ncontinued to rise, while in the Mediterranean premium\nvalues were steady on the week as refiners held their ground\namid rising outright values and slowing demand.The Asia-Pacific bitumen market remained mostly firm,\ndespite the expected slowdown due to the week-long break\nin China for Golden Week holidays.The continued buoyancy in crude and HSFO prices in the\nAsia-Pacific region pushed refiners to either seek price hikes\nor mull production cuts at their units. Supply for October\nwas seen to be balanced in key export expo

In [35]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import (
    create_stuff_documents_chain,
)
from langchain_core.documents import Document

# 요약을 위한 프롬프트 템플릿을 정의합니다.
prompt = PromptTemplate.from_template(
    """Please summarize the sentence according to the following REQUEST.
    
REQUEST:
1. Summarize the main points in bullet points.
2. Write the summary in same language as the context.
3. DO NOT translate any technical terms.
4. DO NOT include any unnecessary information.
5. Summary must include important entities, numerical values.

CONTEXT:
{context}

SUMMARY:"
"""
)

# ChatOpenAI 모델의 또 다른 인스턴스를 생성합니다. (이전 인스턴스와 동일한 설정)
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
)

# 문서 요약을 위한 체인을 생성합니다.
# 이 체인은 여러 문서를 입력받아 하나의 요약된 텍스트로 결합합니다.
text_summary_chain = create_stuff_documents_chain(llm, prompt)

In [36]:
def create_text_summary(state: GraphState):
    # state에서 텍스트 데이터를 가져옵니다.
    texts = state["texts"]

    # 요약된 텍스트를 저장할 딕셔너리를 초기화합니다.
    text_summary = dict()

    # texts.items()를 페이지 번호(키)를 기준으로 오름차순 정렬합니다.
    sorted_texts = sorted(texts.items(), key=lambda x: x[0])

    # 각 페이지의 텍스트를 Document 객체로 변환하여 입력 리스트를 생성합니다.
    inputs = [
        {"context": [Document(page_content=text)]} for page_num, text in sorted_texts
    ]

    # text_summary_chain을 사용하여 일괄 처리로 요약을 생성합니다.
    summaries = text_summary_chain.batch(inputs)

    # 생성된 요약을 페이지 번호와 함께 딕셔너리에 저장합니다.
    for page_num, summary in enumerate(summaries):
        text_summary[page_num] = summary

    # 요약된 텍스트를 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(text_summary=text_summary)


# create_text_summary 함수를 호출하여 텍스트 요약을 생성합니다.
state_out = create_text_summary(state)

# 생성된 요약을 기존 state에 업데이트합니다.
state.update(state_out)

# 요약된 텍스트를 출력합니다.
state_out["text_summary"]

{0: "- Bitumen cargo values increased in Europe and Asia-Pacific due to rising crude and fuel oil prices.\n- Domestic truck prices in Europe mostly strengthened at the start of October.\n- Cargo premiums to high-sulphur fuel oil (HSFO) in Rotterdam and the Baltics decreased by $2-3/t, but outright values continued to rise.\n- Mediterranean premium values remained steady amid rising outright values and slowing demand.\n- The Asia-Pacific bitumen market remained firm despite a slowdown expected during China's Golden Week holidays.\n- Crude and HSFO prices in Asia-Pacific led refiners to consider price hikes or production cuts.\n- Supply for October was balanced in key export locations like Singapore and South Korea due to lower bitumen production.\n- Demand is anticipated to improve from Vietnam, Australia, China, and India due to seasonal factors and easing pandemic restrictions.\n- Bitumen prices surged in sub-Saharan Africa, with higher delivered cargo and drummed import prices, as we

## Image, Table 요약을 위한 데이터 배치 생성

In [37]:
state["page_elements"][2]["image_elements"][0]

{'bounding_box': [{'x': 90, 'y': 1229},
  {'x': 598, 'y': 1229},
  {'x': 598, 'y': 1557},
  {'x': 90, 'y': 1557}],
 'category': 'figure',
 'html': '<figure><img id=\'52\' style=\'font-size:14px\' alt="Southern UK domestic Rotterdam domestic\n550\n500\n450 hhhhhhhhhhh\n400\n350\n22 Jan 16 Apr 9 Jul 1 Oct" data-coord="top-left:(89,1228); bottom-right:(597,1557)" /></figure>',
 'id': 52,
 'page': 2,
 'text': 'Southern UK domestic Rotterdam domestic\n550\n500\n450 hhhhhhhhhhh\n400\n350\n22 Jan 16 Apr 9 Jul 1 Oct'}

In [38]:
print(state["text_summary"][2])

- Domestic truck prices increased in several markets due to October supply revisions.
- German domestic prices rose by €10/t; Benelux, UK, and France saw gains of €20/t, £20/t, and €20-25/t respectively.
- Not all monthly deals were confirmed by 1 October; further details expected on 4 October.
- Czech domestic and export prices strengthened due to tight refinery supply, despite overall plentiful availability in central and southeast Europe.
- High-sulphur fuel oil (HSFO) price increases in late September raised bitumen values, but early winter signs led to a modest $2-3/t decrease in Rotterdam and Baltic cargo export premiums, assessed at $25-30/t and around $20/t fob respectively.
- Cross regional freight rates for 5,000t cargoes showed slight declines: £22-24/t from Rotterdam, £29-31/t from Hamburg, £47-49/t from Klaipeda, and £39-43/t from La Coruna.
- UK domestic truck prices were assessed £20/t higher at £380-390/t ex-works and £395-405/t delivered after October price hikes.


In [39]:
def create_image_summary_data_batches(state: GraphState):
    # 이미지 요약을 위한 데이터 배치를 생성하는 함수
    data_batches = []

    # 페이지 번호를 오름차순으로 정렬
    page_numbers = sorted(list(state["page_elements"].keys()))

    for page_num in page_numbers:
        # 각 페이지의 요약된 텍스트를 가져옴
        text = state["text_summary"][page_num]
        # 해당 페이지의 모든 이미지 요소에 대해 반복
        for image_element in state["page_elements"][page_num]["image_elements"]:
            # 이미지 ID를 정수로 변환
            image_id = int(image_element["id"])

            # 데이터 배치에 이미지 정보, 관련 텍스트, 페이지 번호, ID를 추가
            data_batches.append(
                {
                    "image": state["images"][image_id],  # 이미지 파일 경로
                    "text": text,  # 관련 텍스트 요약
                    "page": page_num,  # 페이지 번호
                    "id": image_id,  # 이미지 ID
                }
            )
    # 생성된 데이터 배치를 GraphState 객체에 담아 반환
    return GraphState(image_summary_data_batches=data_batches)


def create_table_summary_data_batches(state: GraphState):
    # 테이블 요약을 위한 데이터 배치를 생성하는 함수
    data_batches = []

    # 페이지 번호를 오름차순으로 정렬
    page_numbers = sorted(list(state["page_elements"].keys()))

    for page_num in page_numbers:
        # 각 페이지의 요약된 텍스트를 가져옴
        text = state["text_summary"][page_num]
        # 해당 페이지의 모든 테이블 요소에 대해 반복
        for image_element in state["page_elements"][page_num]["table_elements"]:
            # 테이블 ID를 정수로 변환
            image_id = int(image_element["id"])

            # 데이터 배치에 테이블 정보, 관련 텍스트, 페이지 번호, ID를 추가
            data_batches.append(
                {
                    "table": state["tables"][image_id],  # 테이블 데이터
                    "text": text,  # 관련 텍스트 요약
                    "page": page_num,  # 페이지 번호
                    "id": image_id,  # 테이블 ID
                }
            )
    # 생성된 데이터 배치를 GraphState 객체에 담아 반환
    return GraphState(table_summary_data_batches=data_batches)

In [40]:
state_out = create_image_summary_data_batches(state)
state.update(state_out)
state_out

{'image_summary_data_batches': [{'image': 'data/sample-report/15.png',
   'text': "- Bitumen cargo values increased in Europe and Asia-Pacific due to rising crude and fuel oil prices.\n- Domestic truck prices in Europe mostly strengthened at the start of October.\n- Cargo premiums to high-sulphur fuel oil (HSFO) in Rotterdam and the Baltics decreased by $2-3/t, but outright values continued to rise.\n- Mediterranean premium values remained steady amid rising outright values and slowing demand.\n- The Asia-Pacific bitumen market remained firm despite a slowdown expected during China's Golden Week holidays.\n- Crude and HSFO prices in Asia-Pacific led refiners to consider price hikes or production cuts.\n- Supply for October was balanced in key export locations like Singapore and South Korea due to lower bitumen production.\n- Demand is anticipated to improve from Vietnam, Australia, China, and India due to seasonal factors and easing pandemic restrictions.\n- Bitumen prices surged in su

In [41]:
state_out = create_table_summary_data_batches(state)
state.update(state_out)
state_out

{'table_summary_data_batches': [{'table': 'data/sample-report/12.png',
   'text': "- Bitumen cargo values increased in Europe and Asia-Pacific due to rising crude and fuel oil prices.\n- Domestic truck prices in Europe mostly strengthened at the start of October.\n- Cargo premiums to high-sulphur fuel oil (HSFO) in Rotterdam and the Baltics decreased by $2-3/t, but outright values continued to rise.\n- Mediterranean premium values remained steady amid rising outright values and slowing demand.\n- The Asia-Pacific bitumen market remained firm despite a slowdown expected during China's Golden Week holidays.\n- Crude and HSFO prices in Asia-Pacific led refiners to consider price hikes or production cuts.\n- Supply for October was balanced in key export locations like Singapore and South Korea due to lower bitumen production.\n- Demand is anticipated to improve from Vietnam, Australia, China, and India due to seasonal factors and easing pandemic restrictions.\n- Bitumen prices surged in su

## Image, Table 요약 추출

In [42]:
from langchain_teddynote.models import MultiModal
from langchain_core.runnables import chain


@chain
def extract_image_summary(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = """You are an expert in extracting useful information from IMAGE.
    With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval."""

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"]
        image_path = data_batch["image"]
        user_prompt_template = f"""Here is the context related to the image: {context}
        
###

Output Format:

<image>
<title>
<summary>
<entities> 
</image>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer


@chain
def extract_table_summary(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = "You are an expert in extracting useful information from TABLE. With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval."

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"]
        image_path = data_batch["table"]
        user_prompt_template = f"""Here is the context related to the image of table: {context}
        
###

Output Format:

<table>
<title>
<table_summary>
<key_entities> 
<data_insights>
</table>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer

In [43]:
def create_image_summary(state: GraphState):
    # 이미지 요약 추출
    # extract_image_summary 함수를 호출하여 이미지 요약 생성
    image_summaries = extract_image_summary.invoke(
        state["image_summary_data_batches"],
    )

    # 이미지 요약 결과를 저장할 딕셔너리 초기화
    image_summary_output = dict()

    # 각 데이터 배치와 이미지 요약을 순회하며 처리
    for data_batch, image_summary in zip(
        state["image_summary_data_batches"], image_summaries
    ):
        # 데이터 배치의 ID를 키로 사용하여 이미지 요약 저장
        image_summary_output[data_batch["id"]] = image_summary

    # 이미지 요약 결과를 포함한 새로운 GraphState 객체 반환
    return GraphState(image_summary=image_summary_output)


# 이미지 요약 생성 함수 실행
state_out = create_image_summary(state)

# 기존 상태 업데이트
state.update(state_out)

# 이미지 요약 결과 출력
state_out["image_summary"]

{15: '<image>\n<title>Bitumen Price Trends (January - October)</title>\n<summary>This line graph illustrates the price trends of bitumen from January to October. The prices show a general upward trend, with fluctuations observed throughout the year. Notable increases are evident as the year progresses, particularly leading up to October.</summary>\n<entities>\n- Bitumen\n- Price trends\n- Europe\n- Asia-Pacific\n- Crude oil\n- High-sulphur fuel oil (HSFO)\n- Rotterdam\n- Baltics\n- Mediterranean\n- Vietnam\n- Australia\n- China\n- India\n- Sub-Saharan Africa\n- South Africa\n</entities>\n</image>',
 23: "<image>\n<title>Global Shipping Rates and Export Destinations</title>\n<summary>This map illustrates various shipping rates per ton from different regions, highlighting Spain's export rate of $460/t. It also shows other regions such as Rotterdam, Italy, and the Ivory Coast with their respective rates.</summary>\n<entities>\n- Regions: Spain, Rotterdam, Italy, Baltic, Greece, South Kore

In [44]:
def create_table_summary(state: GraphState):
    # 테이블 요약 추출
    table_summaries = extract_table_summary.invoke(
        state["table_summary_data_batches"],
    )

    # 테이블 요약 결과를 저장할 딕셔너리 초기화
    table_summary_output = dict()

    # 각 데이터 배치와 테이블 요약을 순회하며 처리
    for data_batch, table_summary in zip(
        state["table_summary_data_batches"], table_summaries
    ):
        # 데이터 배치의 ID를 키로 사용하여 테이블 요약 저장
        table_summary_output[data_batch["id"]] = table_summary

    # 테이블 요약 결과를 포함한 새로운 GraphState 객체 반환
    return GraphState(table_summary=table_summary_output)


# 테이블 요약 생성 함수 실행
state_out = create_table_summary(state)

# 기존 상태 업데이트
state.update(state_out)

# 테이블 요약 결과 출력
state_out["table_summary"]

{12: "<table>\n<title>Bitumen Prices Summary (25 Sep - 1 Oct)</title>\n<table_summary>Summary of bitumen prices across key locations, highlighting export cargo and domestic prices.</table_summary>\n<key_entities>\n- Export Cargo Prices (fob)\n  - Mediterranean: $448.58 - $453.58\n  - Rotterdam: $460.90 - $465.90\n  - Baltic: $453.90 - $457.90\n  - Singapore: $405 - $415\n  - South Korea: $395 - $405\n  - Mideast Gulf: $298 - $425\n- Delivered Cargo Prices (cfr)\n  - North Africa (Alexandria, bulk): $486 - $496\n  - East Africa (Mombasa, drum): $465 - $475\n  - West Africa (Lagos, bulk): $600 - $610\n  - East China coast: $425 - $475\n- Domestic Prices\n  - Antwerp (ex-works): $490 - $507\n  - Southern Germany (ex-works): $431 - $449\n  - Hungary (ex-works): $443 - $455\n  - Italy (ex-works inc tax): $455 - $466\n  - Indonesia (ex-works): $444\n  - Mumbai (bulk): $544 - $594\n</key_entities>\n<data_insights>\n- Bitumen prices have generally increased in Europe and Asia-Pacific due to ri

## Table Markdown 추출

In [46]:
@chain
def table_markdown_extractor(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = "You are an expert in converting image of the TABLE into markdown format. Be sure to include all the information in the table. DO NOT narrate, just answer in markdown format."

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        image_path = data_batch["table"]
        user_prompt_template = f"""DO NOT wrap your answer in ```markdown``` or any XML tags.
        
###

Output Format:

<table_markdown>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer

In [47]:
def create_table_markdown(state: GraphState):
    # table_markdown_extractor를 사용하여 테이블 마크다운 생성
    # state["table_summary_data_batches"]에 저장된 테이블 데이터를 사용
    table_markdowns = table_markdown_extractor.invoke(
        state["table_summary_data_batches"],
    )

    # 결과를 저장할 딕셔너리 초기화
    table_markdown_output = dict()

    # 각 데이터 배치와 생성된 테이블 마크다운을 매칭하여 저장
    for data_batch, table_summary in zip(
        state["table_summary_data_batches"], table_markdowns
    ):
        # 데이터 배치의 id를 키로 사용하여 테이블 마크다운 저장
        table_markdown_output[data_batch["id"]] = table_summary

    # 새로운 GraphState 객체 반환, table_markdown 키에 결과 저장
    return GraphState(table_markdown=table_markdown_output)


# create_table_markdown 함수 실행
state_out = create_table_markdown(state)

# 기존 state 업데이트
state.update(state_out)

# 생성된 테이블 마크다운 출력
state_out["table_markdown"]

{12: '| Location                     | Low    | High   | ±       |\n|------------------------------|--------|--------|---------|\n| **Export cargo prices fob**  |        |        |         |\n| Mediterranean                 | 448.58 | 453.58 | +20.93  |\n| Rotterdam                     | 460.90 | 465.90 | +18.95  |\n| Baltic                        | 453.90 | 457.90 | +18.95  |\n| Singapore                     | 405    | 415    | +5.00   |\n| South Korea                   | 395    | 405    | nc      |\n| Mideast Gulf                  | 298    | 425    | +5.50   |\n| **Delivered cargo prices cfr**|        |        |         |\n| North Africa                  | 486    | 496    | +18.00  |\n| East Africa                   | 465    | 475    | +26.00  |\n| West Africa                   | 600    | 610    | +21.00  |\n| East China coast              | 425    | 475    | +2.50   |\n| **Domestic prices**          |        |        |         |\n| Antwerp                       | 490    | 507    | +

In [49]:
print(state_out["table_markdown"][12])

| Location                     | Low    | High   | ±       |
|------------------------------|--------|--------|---------|
| **Export cargo prices fob**  |        |        |         |
| Mediterranean                 | 448.58 | 453.58 | +20.93  |
| Rotterdam                     | 460.90 | 465.90 | +18.95  |
| Baltic                        | 453.90 | 457.90 | +18.95  |
| Singapore                     | 405    | 415    | +5.00   |
| South Korea                   | 395    | 405    | nc      |
| Mideast Gulf                  | 298    | 425    | +5.50   |
| **Delivered cargo prices cfr**|        |        |         |
| North Africa                  | 486    | 496    | +18.00  |
| East Africa                   | 465    | 475    | +26.00  |
| West Africa                   | 600    | 610    | +21.00  |
| East China coast              | 425    | 475    | +2.50   |
| **Domestic prices**          |        |        |         |
| Antwerp                       | 490    | 507    | +20.50  |
| Southern G

## 결과물

In [53]:
state.keys()

dict_keys(['filepath', 'batch_size', 'split_filepaths', 'analyzed_files', 'page_metadata', 'page_elements', 'page_numbers', 'images', 'tables', 'texts', 'text_summary', 'image_summary_data_batches', 'table_summary_data_batches', 'image_summary', 'table_summary', 'table_markdown'])

In [54]:
state["table_summary"]

{12: "<table>\n<title>Bitumen Prices Summary (25 Sep - 1 Oct)</title>\n<table_summary>Summary of bitumen prices across key locations, highlighting export cargo and domestic prices.</table_summary>\n<key_entities>\n- Export Cargo Prices (fob)\n  - Mediterranean: $448.58 - $453.58\n  - Rotterdam: $460.90 - $465.90\n  - Baltic: $453.90 - $457.90\n  - Singapore: $405 - $415\n  - South Korea: $395 - $405\n  - Mideast Gulf: $298 - $425\n- Delivered Cargo Prices (cfr)\n  - North Africa (Alexandria, bulk): $486 - $496\n  - East Africa (Mombasa, drum): $465 - $475\n  - West Africa (Lagos, bulk): $600 - $610\n  - East China coast: $425 - $475\n- Domestic Prices\n  - Antwerp (ex-works): $490 - $507\n  - Southern Germany (ex-works): $431 - $449\n  - Hungary (ex-works): $443 - $455\n  - Italy (ex-works inc tax): $455 - $466\n  - Indonesia (ex-works): $444\n  - Mumbai (bulk): $544 - $594\n</key_entities>\n<data_insights>\n- Bitumen prices have generally increased in Europe and Asia-Pacific due to ri

In [55]:
state["table_markdown"]

{12: '| Location                     | Low    | High   | ±       |\n|------------------------------|--------|--------|---------|\n| **Export cargo prices fob**  |        |        |         |\n| Mediterranean                 | 448.58 | 453.58 | +20.93  |\n| Rotterdam                     | 460.90 | 465.90 | +18.95  |\n| Baltic                        | 453.90 | 457.90 | +18.95  |\n| Singapore                     | 405    | 415    | +5.00   |\n| South Korea                   | 395    | 405    | nc      |\n| Mideast Gulf                  | 298    | 425    | +5.50   |\n| **Delivered cargo prices cfr**|        |        |         |\n| North Africa                  | 486    | 496    | +18.00  |\n| East Africa                   | 465    | 475    | +26.00  |\n| West Africa                   | 600    | 610    | +21.00  |\n| East China coast              | 425    | 475    | +2.50   |\n| **Domestic prices**          |        |        |         |\n| Antwerp                       | 490    | 507    | +

In [56]:
state["image_summary"]

{15: '<image>\n<title>Bitumen Price Trends (January - October)</title>\n<summary>This line graph illustrates the price trends of bitumen from January to October. The prices show a general upward trend, with fluctuations observed throughout the year. Notable increases are evident as the year progresses, particularly leading up to October.</summary>\n<entities>\n- Bitumen\n- Price trends\n- Europe\n- Asia-Pacific\n- Crude oil\n- High-sulphur fuel oil (HSFO)\n- Rotterdam\n- Baltics\n- Mediterranean\n- Vietnam\n- Australia\n- China\n- India\n- Sub-Saharan Africa\n- South Africa\n</entities>\n</image>',
 23: "<image>\n<title>Global Shipping Rates and Export Destinations</title>\n<summary>This map illustrates various shipping rates per ton from different regions, highlighting Spain's export rate of $460/t. It also shows other regions such as Rotterdam, Italy, and the Ivory Coast with their respective rates.</summary>\n<entities>\n- Regions: Spain, Rotterdam, Italy, Baltic, Greece, South Kore

In [57]:
state["text_summary"]

{0: "- Bitumen cargo values increased in Europe and Asia-Pacific due to rising crude and fuel oil prices.\n- Domestic truck prices in Europe mostly strengthened at the start of October.\n- Cargo premiums to high-sulphur fuel oil (HSFO) in Rotterdam and the Baltics decreased by $2-3/t, but outright values continued to rise.\n- Mediterranean premium values remained steady amid rising outright values and slowing demand.\n- The Asia-Pacific bitumen market remained firm despite a slowdown expected during China's Golden Week holidays.\n- Crude and HSFO prices in Asia-Pacific led refiners to consider price hikes or production cuts.\n- Supply for October was balanced in key export locations like Singapore and South Korea due to lower bitumen production.\n- Demand is anticipated to improve from Vietnam, Australia, China, and India due to seasonal factors and easing pandemic restrictions.\n- Bitumen prices surged in sub-Saharan Africa, with higher delivered cargo and drummed import prices, as we

In [58]:
state["texts"]

{0: 'Argus BitumenEurope, Africa, Middle East and Asia-Pacific prices and commentary\nIncorporating Argus Asphalt ReportIssue 21-39 | Friday 1 October 2021SummaryBitumen cargo values gained across Europe and Asia-Pacific,\nboosted by a jump in crude and fuel oil values.Domestic truck prices across Europe also mostly\nstrengthened as October started.Cargo premiums to high-sulphur fuel oil (HSFO) in Rot-\nterdam and the Baltics fell by $2-3/t, but outright values\ncontinued to rise, while in the Mediterranean premium\nvalues were steady on the week as refiners held their ground\namid rising outright values and slowing demand.The Asia-Pacific bitumen market remained mostly firm,\ndespite the expected slowdown due to the week-long break\nin China for Golden Week holidays.The continued buoyancy in crude and HSFO prices in the\nAsia-Pacific region pushed refiners to either seek price hikes\nor mull production cuts at their units. Supply for October\nwas seen to be balanced in key export expo

In [59]:
state["text_summary"]

{0: "- Bitumen cargo values increased in Europe and Asia-Pacific due to rising crude and fuel oil prices.\n- Domestic truck prices in Europe mostly strengthened at the start of October.\n- Cargo premiums to high-sulphur fuel oil (HSFO) in Rotterdam and the Baltics decreased by $2-3/t, but outright values continued to rise.\n- Mediterranean premium values remained steady amid rising outright values and slowing demand.\n- The Asia-Pacific bitumen market remained firm despite a slowdown expected during China's Golden Week holidays.\n- Crude and HSFO prices in Asia-Pacific led refiners to consider price hikes or production cuts.\n- Supply for October was balanced in key export locations like Singapore and South Korea due to lower bitumen production.\n- Demand is anticipated to improve from Vietnam, Australia, China, and India due to seasonal factors and easing pandemic restrictions.\n- Bitumen prices surged in sub-Saharan Africa, with higher delivered cargo and drummed import prices, as we